In [1]:
!pip install --quiet datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
import pandas as pd
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import os
from accelerate import Accelerator
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import datasets

In [11]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-large-emotion-latest", device_map="cuda:0")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-large-emotion-latest", device_map="cuda:0")
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [12]:
# Freezing the base model
for param in model.parameters():
    param.requires_grad = False
# Unfreeze the last classifier layer
for param in model.classifier.parameters():
    param.requires_grad = True

In [38]:
df = pd.read_csv('/content/eng.csv')

In [39]:
type(df.iloc[0]['Anger'])

numpy.int64

In [40]:
# # List of emotions in the GoEmotions dataset
# emotion_labels = [
#     "admiration", "amusement", "anger", "annoyance", "approval",
#     "caring", "confusion", "curiosity", "desire", "disappointment",
#     "disapproval", "disgust", "embarrassment", "excitement", "fear",
#     "gratitude", "grief", "joy", "love", "nervousness",
#     "optimism", "pride", "realization", "relief", "remorse",
#     "sadness", "surprise", "neutral"
# ]

# # Example: Finding the emotion name for label number 3
# label_number = 3  # Using 0-based index for label number
# emotion_name = emotion_labels[label_number]
# print("The emotion is:", emotion_name)

# Setting the labels of our dataset in the same sequence as above
df['labels'] = df.apply(lambda row:[float(row['Anger']), float(row['Fear']), float(row['Joy']), float(row['Sadness']), float(row['Surprise'])], axis=1)
df = df.drop(columns=['id','Joy','Fear','Anger','Sadness','Surprise'])
df

,text,labels
0,But not very happy.,"[0.0, 0.0, 1.0, 1.0, 0.0]"
1,Well she's not gon na last the whole song like...,"[0.0, 0.0, 1.0, 0.0, 0.0]"
2,She sat at her Papa's recliner sofa only to mo...,"[0.0, 0.0, 0.0, 0.0, 0.0]"
3,"Yes, the Oklahoma city bombing.","[1.0, 1.0, 0.0, 1.0, 1.0]"
4,They were dancing to Bolero.,"[0.0, 0.0, 1.0, 0.0, 0.0]"
...,...,...
2763,"""Yeah, but did you just find that?","[0.0, 1.0, 0.0, 0.0, 1.0]"
2764,I did as little as possible with my right hand...,"[0.0, 0.0, 0.0, 0.0, 0.0]"
2765,"Okay that sucks, right?","[1.0, 0.0, 0.0, 1.0, 0.0]"
2766,"The spark leaped through his body into mine, a...","[0.0, 1.0, 0.0, 0.0, 1.0]"


In [41]:
emotion_labels = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']

# Create a new dataframe to hold the transformed data
transformed_data = []

# Iterate through each row in the original dataframe
for _, row in df.iterrows():
    text = row['text']
    labels = row['labels']

    # For each emotion, create a new row
    for i, label in enumerate(labels):
        if label == 1.0:
            transformed_data.append({
                'premise': text,
                'hypothesis': f"The speaker is feeling {emotion_labels[i]}",
                'label': [1.0]  # The emotion is present in the premise
            })
        else:
            transformed_data.append({
                'premise': text,
                'hypothesis': f"The speaker is feeling {emotion_labels[i]}",
                'label': [0.0]  # The emotion is not present in the premise
            })

# Convert the transformed data into a new DataFrame
transformed_df = pd.DataFrame(transformed_data)

# Display the transformed DataFrame
print(transformed_df)

                                                 premise  \
0                                    But not very happy.   
1                                    But not very happy.   
2                                    But not very happy.   
3                                    But not very happy.   
4                                    But not very happy.   
...                                                  ...   
13835  He had 4 inches and 40 pounds on me and I stil...   
13836  He had 4 inches and 40 pounds on me and I stil...   
13837  He had 4 inches and 40 pounds on me and I stil...   
13838  He had 4 inches and 40 pounds on me and I stil...   
13839  He had 4 inches and 40 pounds on me and I stil...   

                            hypothesis  label  
0         The speaker is feeling Anger  [0.0]  
1          The speaker is feeling Fear  [0.0]  
2           The speaker is feeling Joy  [1.0]  
3       The speaker is feeling Sadness  [1.0]  
4      The speaker is feeling Surprise 

In [42]:
from datasets import Dataset

dataset = Dataset.from_pandas(transformed_df)  # Use only the desired columns

# Optionally, print the dataset to check
print(dataset)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 13840
})


In [43]:
def preprocess_function(examples):
    # Tokenize the text
    inputs = tokenizer(examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", max_length=384)

    # Convert labels to torch float to match the expected input type for BCEWithLogitsLoss
    inputs["labels"] = torch.tensor(examples["label"])

    return inputs

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset

Map:   0%|          | 0/13840 [00:00<?, ? examples/s]

Dataset({
    features: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 13840
})

In [44]:
print(encoded_dataset[0]["labels"])

[0.0]


In [45]:
encoded_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [46]:
print(encoded_dataset[0]["labels"])  # Should output a tensor of floats: e.g., tensor([1., 0., 0., ..., 0., 1.])


tensor([0.])


In [47]:
dataset = encoded_dataset.train_test_split(test_size=0.2)

In [48]:
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11072
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2768
    })
})

In [49]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Sigmoid is typically used for multi-label classification to get probabilities
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    # Convert probabilities to binary predictions (0 or 1)
    predictions = (probs > 0.5).astype(int)

    # Calculate metrics
    f1 = f1_score(labels, predictions, average='macro', zero_division=1)
    precision = precision_score(labels, predictions, average='macro', zero_division=1)
    recall = recall_score(labels, predictions, average='macro', zero_division=1)
    accuracy = accuracy_score(labels, predictions)  # Multi-label accuracy

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

In [50]:
class ModifiedModel(torch.nn.Module):
    def __init__(self, original_model):
        super(ModifiedModel, self).__init__()
        self.original_model = original_model
        self.fc = torch.nn.Linear(11, 1)  # New FC layer to map 11 -> 1 labels

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.original_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Shape: [batch_size, 11]
        logits = self.fc(logits)  # Shape: [batch_size, 1]
        if labels is not None:
            loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, labels.float())
            return {"loss": loss, "logits": logits}

        return {"logits": logits}

modified_model = ModifiedModel(model)

In [51]:
from transformers import TrainingArguments, Trainer
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="no",
    save_steps=0,
    logging_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"   # Disable wandb logging if not needed
)

# Initialize Trainer
trainer = Trainer(
    model=modified_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

In [52]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.504800,0.462974,0.770231,0.715507,0.721969,0.710424
2,0.467900,0.455680,0.771676,0.740012,0.731566,0.756936
3,0.446100,0.466313,0.785766,0.754848,0.746033,0.770871
4,0.435200,0.429351,0.800939,0.766806,0.760351,0.775464
5,0.422700,0.430764,0.790824,0.759784,0.751044,0.774809


TrainOutput(global_step=6920, training_loss=0.45533129741690753, metrics={'train_runtime': 5099.2923, 'train_samples_per_second': 10.856, 'train_steps_per_second': 1.357, 'total_flos': 0.0, 'train_loss': 0.45533129741690753, 'epoch': 5.0})

In [ ]:
from transformers import TrainingArguments, Trainer
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="no",
    save_steps=0,
    logging_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"   # Disable wandb logging if not needed
)

# Initialize Trainer
trainer = Trainer(
    model=modified_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)
# Train the model
trainer.train()

##Using the trained model to get predictions on the train and dev sets

In [53]:
df_val = pd.read_csv('/content/eng_a.csv')
df_val

,id,text,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_a_00001,"My mouth fell open `` No, no, no... I..",NaN,NaN,NaN,NaN,NaN
1,eng_dev_track_a_00002,You can barely make out your daughter's pale f...,NaN,NaN,NaN,NaN,NaN
2,eng_dev_track_a_00003,But after blinking my eyes for a few times lep...,NaN,NaN,NaN,NaN,NaN
3,eng_dev_track_a_00004,Slowly rising to my feet I came to the conclus...,NaN,NaN,NaN,NaN,NaN
4,eng_dev_track_a_00005,I noticed this months after moving in and doin...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,"""ARcH stop your progression.",NaN,NaN,NaN,NaN,NaN
112,eng_dev_track_a_00113,"This 'star', starts to move across the sky.",NaN,NaN,NaN,NaN,NaN
113,eng_dev_track_a_00114,and my feet hurt.,NaN,NaN,NaN,NaN,NaN
114,eng_dev_track_a_00115,so i cried my eyes out and did the drawing.,NaN,NaN,NaN,NaN,NaN


In [54]:
df_val = df_val.drop(columns=['id','Joy','Fear','Anger','Sadness','Surprise'])
df_val

,text
0,"My mouth fell open `` No, no, no... I.."
1,You can barely make out your daughter's pale f...
2,But after blinking my eyes for a few times lep...
3,Slowly rising to my feet I came to the conclus...
4,I noticed this months after moving in and doin...
...,...
111,"""ARcH stop your progression."
112,"This 'star', starts to move across the sky."
113,and my feet hurt.
114,so i cried my eyes out and did the drawing.


In [56]:
emotion_labels = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']

# Create a new dataframe to hold the transformed data
transformed_data_val = []

# Iterate through each row in the original dataframe
for _, row in df_val.iterrows():
    text = row['text']
    # labels = row['labels']

    # For each emotion, create a new row
    for label in emotion_labels:
        # if label == 1.0:
          transformed_data_val.append({
              'premise': text,
              'hypothesis': f"The speaker is feeling {label}",
              # 'label': [1.0]  # The emotion is present in the premise
          })
        # else:
        #     transformed_data.append({
        #         'premise': text,
        #         'hypothesis': f"The speaker is feeling {emotion_labels[i]}",
        #         'label': [0.0]  # The emotion is not present in the premise
        #     })

# Convert the transformed data into a new DataFrame
transformed_data_val = pd.DataFrame(transformed_data_val)

# Display the transformed DataFrame
transformed_data_val

,premise,hypothesis
0,"My mouth fell open `` No, no, no... I..",The speaker is feeling Anger
1,"My mouth fell open `` No, no, no... I..",The speaker is feeling Fear
2,"My mouth fell open `` No, no, no... I..",The speaker is feeling Joy
3,"My mouth fell open `` No, no, no... I..",The speaker is feeling Sadness
4,"My mouth fell open `` No, no, no... I..",The speaker is feeling Surprise
...,...,...
575,They were coal black.,The speaker is feeling Anger
576,They were coal black.,The speaker is feeling Fear
577,They were coal black.,The speaker is feeling Joy
578,They were coal black.,The speaker is feeling Sadness


In [57]:
val_dataset = Dataset.from_pandas(transformed_data_val)  # Use only the desired columns

# Optionally, print the dataset to check
print(val_dataset)

Dataset({
    features: ['premise', 'hypothesis'],
    num_rows: 580
})


In [58]:
def preprocess_function_val(examples):
    # Tokenize the text
    inputs = tokenizer(examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", max_length=384)

    # Convert labels to torch float to match the expected input type for BCEWithLogitsLoss
    # inputs["labels"] = [torch.tensor(label, dtype=torch.float) for label in examples["labels"]]

    return inputs

encoded_dataset_val = val_dataset.map(preprocess_function_val, batched=True)
encoded_dataset_val

Map:   0%|          | 0/580 [00:00<?, ? examples/s]

Dataset({
    features: ['premise', 'hypothesis', 'input_ids', 'attention_mask'],
    num_rows: 580
})

In [59]:
encoded_dataset_val.set_format("torch", columns=["input_ids", "attention_mask"])
encoded_dataset_val

Dataset({
    features: ['premise', 'hypothesis', 'input_ids', 'attention_mask'],
    num_rows: 580
})

In [60]:
val_training_args = TrainingArguments(
    output_dir="./results",
    # per_device_eval_batch_size=8,
    # num_train_epochs=80,
    # weight_decay=0.01,
    report_to="none"   # Disable wandb logging if not needed
)
val_trainer = Trainer(
    model=modified_model,  # model is your trained model
    args=val_training_args,
    tokenizer=tokenizer
)

# Run predictions
predictions = val_trainer.predict(encoded_dataset_val)
pred_logits = predictions.predictions

In [61]:
pred_labels = (torch.sigmoid(torch.tensor(pred_logits)) > 0.5).int()

In [63]:
pred_labels.shape

torch.Size([580, 1])

In [66]:
predicted_labels = pred_labels  # Replace with your actual tensor

# Reshape to group every 5 labels together
final_labels = predicted_labels.view(-1, 5)

# Convert each row to a list format
# final_labels = [row.tolist() for row in predicted_labels_grouped]

# Print the final shape and labels
print(final_labels)  # This will be a list of 116 lists, each containing 5 labels
print(len(final_labels))  # Output should be 116

tensor([[0, 1, 0, 0, 1],
        [0, 1, 0, 1, 1],
        [0, 0, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 1],
        [0, 1, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 1, 0],
        [1, 1, 0, 1, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [1, 1, 0, 0, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 1, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 1, 0],
        [1, 1, 0, 1, 0],
        [1, 1, 0, 1, 1],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 1, 1, 1],
        [0, 0, 1, 1, 0],


In [68]:
df_val = pd.DataFrame(final_labels, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])
df_val

,Anger,Fear,Joy,Sadness,Surprise
0,0,1,0,0,1
1,0,1,0,1,1
2,0,0,0,0,0
3,0,1,0,1,0
4,0,1,0,1,1
...,...,...,...,...,...
111,0,1,0,0,1
112,0,1,1,0,1
113,0,1,0,1,0
114,0,0,0,1,0


In [69]:
df_val.insert(loc=0, column='id', value='text')
df_val['id'] = [f'eng_dev_track_a_{i+1:05}' for i in range(len(df_val))]
df_val

,id,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_a_00001,0,1,0,0,1
1,eng_dev_track_a_00002,0,1,0,1,1
2,eng_dev_track_a_00003,0,0,0,0,0
3,eng_dev_track_a_00004,0,1,0,1,0
4,eng_dev_track_a_00005,0,1,0,1,1
...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0,1,0,0,1
112,eng_dev_track_a_00113,0,1,1,0,1
113,eng_dev_track_a_00114,0,1,0,1,0
114,eng_dev_track_a_00115,0,0,0,1,0


In [70]:
df_val.to_csv('my_pred_after_FCLayer_and_lastlayer_twitterRoberta.csv', index=False)